In [7]:
import os
os.environ['PYTHONHASHSEED']=str(0)

In [8]:
"""
Coin Game environment.
"""
import sys
import gym
import pylab
import random
import numpy as np
import tensorflow as tf
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
from keras import backend as K

from gym.spaces import Discrete, Tuple
from gym.spaces import prng

np.random.seed(123)
random.seed(456)
prng.seed(456)

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)


tf.set_random_seed(789)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)
    


class CoinGameVec:
    """
    Vectorized Coin Game environment.
    Note: slightly deviates from the Gym API.
    """
    NUM_AGENTS = 2
    NUM_ACTIONS = 4
    MOVES = [
        np.array([0,  1]),
        np.array([0, -1]),
        np.array([1,  0]),
        np.array([-1, 0]), 
    ]

    def __init__(self, max_steps = 10, batch_size = 1, grid_size=3):
        self.max_steps = max_steps
        self.grid_size = grid_size
        self.batch_size = batch_size
        # The 4 channels stand for 2 players and 2 coin positions
        self.ob_space_shape = [4, grid_size, grid_size]

        self.step_count = None

    def reset(self):
        self.step_count = 0
        self.red_coin = prng.np_random.randint(2, size=self.batch_size)
        # Agent and coin positions
        self.red_pos  = prng.np_random.randint(
            self.grid_size, size=(self.batch_size, 2))
        self.blue_pos = prng.np_random.randint(
            self.grid_size, size=(self.batch_size, 2))
        self.coin_pos = np.zeros((self.batch_size, 2), dtype=np.int8)
        for i in range(self.batch_size):
            # Make sure coins don't overlap
            while self._same_pos(self.red_pos[i], self.blue_pos[i]):
                self.blue_pos[i] = prng.np_random.randint(self.grid_size, size=2)
            self._generate_coin(i)
        return self._generate_state()

    def _generate_coin(self, i):
        self.red_coin[i] = 1 - self.red_coin[i]
        # Make sure coin has a different position than the agents
        success = 0
        while success < 2:
            self.coin_pos[i] = prng.np_random.randint(self.grid_size, size=(2))
            success  = 1 - self._same_pos(self.red_pos[i],
                                          self.coin_pos[i])
            success += 1 - self._same_pos(self.blue_pos[i],
                                          self.coin_pos[i])

    def _same_pos(self, x, y):
        return (x == y).all()

    def _generate_state(self):
        state = np.zeros([self.batch_size] + self.ob_space_shape)
        for i in range(self.batch_size):
            state[i, 0, self.red_pos[i][0], self.red_pos[i][1]] = 1
            state[i, 1, self.blue_pos[i][0], self.blue_pos[i][1]] = 1
            if self.red_coin[i]:
                state[i, 2, self.coin_pos[i][0], self.coin_pos[i][1]] = 1
            else:
                state[i, 3, self.coin_pos[i][0], self.coin_pos[i][1]] = 1
        return state

    def step(self, action1, action2):
        for j in range(self.batch_size):
            ac0 = action1
            ac1 = action2
            assert ac0 in {0, 1, 2, 3} and ac1 in {0, 1, 2, 3}

            # Move players
            self.red_pos[j] = \
                (self.red_pos[j] + self.MOVES[ac0]) % self.grid_size
            self.blue_pos[j] = \
                (self.blue_pos[j] + self.MOVES[ac1]) % self.grid_size

        # Compute rewards
        reward_red, reward_blue = [], []
        for i in range(self.batch_size):
            generate = False
            if self.red_coin[i]:
                if self._same_pos(self.red_pos[i], self.coin_pos[i]):
                    generate = True
                    reward_red.append(0)
                    reward_blue.append(0)
                elif self._same_pos(self.blue_pos[i], self.coin_pos[i]):
                    generate = True
                    reward_red.append(0)
                    reward_blue.append(1)
                else:
                    reward_red.append(1)
                    reward_blue.append(0)

            else:
                if self._same_pos(self.blue_pos[i], self.coin_pos[i]):
                    generate = True
                    reward_red.append(0)
                    reward_blue.append(0)
                elif self._same_pos(self.red_pos[i], self.coin_pos[i]):
                    generate = True
                    reward_red.append(0)
                    reward_blue.append(1)
                else:
                    reward_red.append(1)
                    reward_blue.append(0)

            #if generate:
                #self._generate_coin(i)

        reward = [np.array(reward_red), np.array(reward_blue)]
        self.step_count += 1
        state = self._generate_state()

        return state, reward, generate

class A2CAgent:
    def __init__(self, state_size, action_size):
        # if you want to see Cartpole learning, then change to True
        self.render = False
        self.load_model = False
        # get size of state and action
        self.state_size = state_size
        self.action_size = action_size
        self.value_size = 1
        #self.action_table = [0,1,-4,4]
        self.lam = 0
        self.alpha = 0.2
        #self.target_state = 11

        # These are hyper parameters for the Policy Gradient
        self.discount_factor = 1
        self.actor_lr = 0.005
        self.critic_lr = 0.005

        # create model for policy network
        self.actor_red = self.build_actor()
        self.actor_blue = self.build_actor()
        self.critic = self.build_critic()
        self.critic2 = self.build_critic2()

        #if self.load_model:
            #self.actor.load_weights("./save_model/cartpole_actor.h5")
            #self.critic.load_weights("./save_model/cartpole_critic.h5")

    # approximate policy and value using Neural Network
    # actor: state is input and probability of each action is output of model
    def build_actor(self):
        actor = Sequential()
        actor.add(Dense(48, input_dim=self.state_size, activation='relu',
                        kernel_initializer='he_uniform'))
        #actor.add(Dense(16, activation='relu',
                        #kernel_initializer='he_uniform'))
        actor.add(Dense(self.action_size, activation='softmax',
                        kernel_initializer='he_uniform'))
        actor.summary()
        # See note regarding crossentropy in cartpole_reinforce.py
        actor.compile(loss='categorical_crossentropy',
                      optimizer=Adam(lr=self.actor_lr))
        return actor

    # critic: state is input and value of state is output of model
    def build_critic(self):
        critic = Sequential()
        critic.add(Dense(48, input_dim=self.state_size, activation='relu',
                         kernel_initializer='he_uniform'))
        #critic.add(Dense(16, activation='relu',
                         #kernel_initializer='he_uniform'))
        critic.add(Dense(self.value_size, activation='linear',
                         kernel_initializer='he_uniform'))
        #critic.summary()
        critic.compile(loss="mse", optimizer=Adam(lr=self.critic_lr))
        return critic
    
    def build_critic2(self):
        critic = Sequential()
        critic.add(Dense(48, input_dim=self.state_size, activation='relu',
                         kernel_initializer='he_uniform'))
        #critic.add(Dense(16, activation='relu',
                         #kernel_initializer='he_uniform'))
        critic.add(Dense(self.value_size, activation='linear',
                         kernel_initializer='he_uniform'))
        #critic.summary()
        critic.compile(loss="mse", optimizer=Adam(lr=self.critic_lr))
        return critic
        

    # using the output of policy network, pick action stochastically
    def get_action(self, state):
        policy1 = self.actor_red.predict(state, batch_size=1).flatten()
        policy2 = self.actor_blue.predict(state,batch_size=1).flatten()
        return np.random.choice(self.action_size, 1, p=policy1)[0],np.random.choice(self.action_size, 1, p=policy2)[0]
    
    def get_maxaction(self,state): #To be updated
        policy1 = self.actor_red.predict(state, batch_size=1).flatten()
        policy2 = self.actor_blue.predict(state,batch_size=1).flatten()
        return np.argmax(policy1),np.argmax(policy2)

    # update policy network every episode
    def train_model(self, state, action1,action2, reward1,reward2, next_state, done):
        target = np.zeros((1, self.value_size))
        advantages_red = np.zeros((1, self.action_size))
        advantages_blue = np.zeros((1, self.action_size))
        target2 = np.zeros((1, self.value_size))     

        value = self.critic.predict(state)[0]
        value2 = self.critic2.predict(state)[0]
        next_value = self.critic.predict(next_state)[0]
        next_value2 = self.critic2.predict(next_state)[0]
        
        reward1 = reward1 + self.lam*(reward2 - self.alpha)

        if done:
            advantages_red[0][action1] = -1*(reward1 - value)
            advantages_blue[0][action2] = -1*(reward1 - value)
            target[0][0] = reward1
            target2[0][0] = reward2
        else:
            advantages_red[0][action1] = -1*(reward1 + self.discount_factor * (next_value) - value)
            advantages_blue[0][action2] = -1*(reward1 + self.discount_factor * (next_value) - value)
            target[0][0] = reward1 + self.discount_factor * next_value
            target2[0][0] = reward2 + self.discount_factor * next_value2
#             if np.where(next_state == 1.0)[1][0] == self.target_state:
#                 target2[0][0] = 0 + self.discount_factor * next_value2
#             else:
#                 target2[0][0] = np.where(next_state == 1.0)[1][0]/10 + self.discount_factor * next_value2            
            

        self.actor_red.fit(state, advantages_red, epochs=1, verbose=0)
        self.actor_blue.fit(state, advantages_blue, epochs=1, verbose=0)
        self.critic.fit(state, target, epochs=1, verbose=0)
        self.critic2.fit(state,target2,epochs = 1, verbose = 0)
        
        #initial_state = np.zeros((1,state_size))
        #initial_state[0][0] = 1
        self.lam = min(max(0,(self.lam + 0.001*(self.critic2.predict(state)[0] - self.alpha))), 10)
        #print(self.lam)
        

if __name__ == "__main__":
    # In case of CartPole-v1, maximum length of episode is 500
    #env = gym.make('CartPole-v1')
    # get size of state and action from environment
    state_size = 4*3*3 #env.observation_space.shape[0]
    action_size = 4 #env.action_space.n
    EPISODES = 10000
    

    # make A2C agent
    agent = A2CAgent(state_size, action_size)
    #agent2 = A2CAgent(state_size,action_size)
    game = CoinGameVec()
    
    scores, episodes = [], []

    for i in range(1):
        done = False
        score = 0
        #state = env.reset()
        #state = np.reshape(state, [1, state_size])
        #rand_state = random.randint(0,15)
        #state = np.zeros((1,state_size))
        #state[0][rand_state] = 1
        #state = np.reshape(state, [1, state_size])
        
        average_penalty = 0
        total_penalty = 0
        store_penalty = np.zeros((1,EPISODES))
        
        average_lag = 0
        total_lag = 0
        store_lag = np.zeros((1,EPISODES))

        
        for e in range(EPISODES):
        #state = env.reset()
            state = game.reset()
            state = np.reshape(state, [1, state_size])
            for time in range(10):
            # env.render()
                action1,action2  = agent.get_action(state)
                
                next_state, reward, done = game.step(action1,action2)
                
            #reward = reward if not done else -10
                next_state = np.reshape(next_state, [1, state_size])
                agent.train_model(state, action1, action2,reward[0][0],reward[1][0],next_state, done)
                #agent2.train_model(state, action2, reward[0][0],reward[1][0],next_state, done)
                
                
                print(np.where(state == 1.0)[1][0],np.where(state == 1.0)[1][1] - 9,int(np.where(state == 1.0)[1][2] % 9),"--",action1, action2,"--",reward[0][0],reward[1][0],"--",np.where(next_state == 1.0)[1][0],np.where(next_state == 1.0)[1][1] - 9,np.where(next_state == 1.0)[1][2] % 9)
                
                #print(agent1.lam,agent2.lam)
                state = next_state
                
                average_penalty = average_penalty + reward[1][0]
                average_lag = average_lag + reward[0][0]
                
                if done:
                #print(state)
                    print("episode: {}/{}, score: {}"
                      .format(e, EPISODES, time))
                    print(agent.lam)
                    #print(agent2.lam)
                    break
                #if len(agent.memory) > batch_size:
                    #agent.replay(batch_size)
            total_penalty = total_penalty + average_penalty        
            store_penalty[0][e] = total_penalty/(e+1) 
            print(store_penalty[0][e])
                
            total_lag = total_lag + average_lag
            store_lag[0][e] = total_lag/(e+1)
                
                #if np.mean(store_penalty[0][e-100:e]) < agent.alpha+0.1:
                        #sys.exit()
                #print(store_penalty[0][e])
            average_penalty = 0
            average_lag = 0
        
            print('*******************************')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 48)                1776      
_________________________________________________________________
dense_26 (Dense)             (None, 4)                 196       
Total params: 1,972
Trainable params: 1,972
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 48)                1776      
_________________________________________________________________
dense_28 (Dense)             (None, 4)                 196       
Total params: 1,972
Trainable params: 1,972
Non-trainable params: 0
_________________________________________________________________
4 6 8 -- 3 1 -- 0 0 -- 1 8 8
episode: 0/10000, score: 0
0
0.0
**********

3 1 8 -- 2 0 -- 1 0 -- 6 2 8
6 2 8 -- 0 0 -- 1 0 -- 7 0 8
7 0 8 -- 0 0 -- 0 1 -- 8 1 8
episode: 33/10000, score: 4
[0.02654441]
0.5882352941176471
*******************************
7 1 5 -- 0 0 -- 1 0 -- 8 2 5
8 2 5 -- 1 0 -- 1 0 -- 7 0 5
7 0 5 -- 2 0 -- 1 0 -- 1 1 5
1 1 5 -- 2 0 -- 1 0 -- 4 2 5
4 2 5 -- 3 0 -- 1 0 -- 1 0 5
1 0 5 -- 0 0 -- 1 0 -- 2 1 5
2 1 5 -- 2 0 -- 0 0 -- 5 2 5
episode: 34/10000, score: 6
[0.03060795]
0.5714285714285714
*******************************
1 7 5 -- 2 0 -- 1 0 -- 4 8 5
4 8 5 -- 3 0 -- 1 0 -- 1 6 5
1 6 5 -- 2 0 -- 1 0 -- 4 7 5
4 7 5 -- 3 0 -- 1 0 -- 1 8 5
1 8 5 -- 0 0 -- 1 0 -- 2 6 5
2 6 5 -- 2 0 -- 0 1 -- 5 7 5
episode: 35/10000, score: 5
[0.03194649]
0.5833333333333334
*******************************
3 6 5 -- 3 0 -- 1 0 -- 0 7 5
0 7 5 -- 0 0 -- 1 0 -- 1 8 5
1 8 5 -- 1 0 -- 1 0 -- 0 6 5
0 6 5 -- 2 0 -- 1 0 -- 3 7 5
3 7 5 -- 3 0 -- 1 0 -- 0 8 5
0 8 5 -- 1 0 -- 1 0 -- 2 6 5
2 6 5 -- 0 0 -- 1 0 -- 0 7 5
0 7 5 -- 2 0 -- 1 0 -- 3 8 5
3 8 5 -- 3 0 -- 1 0 -- 0 6 5

6 8 5 -- 2 0 -- 1 0 -- 0 6 5
0 6 5 -- 2 0 -- 1 0 -- 3 7 5
3 7 5 -- 2 0 -- 1 0 -- 6 8 5
6 8 5 -- 2 0 -- 1 0 -- 0 6 5
0 6 5 -- 2 3 -- 1 0 -- 3 3 5
3 3 5 -- 2 0 -- 1 0 -- 6 4 5
6 4 5 -- 3 0 -- 0 0 -- 3 5 5
episode: 71/10000, score: 6
[0.07108648]
0.4722222222222222
*******************************
2 8 1 -- 2 0 -- 1 0 -- 5 6 1
5 6 1 -- 2 0 -- 1 0 -- 8 7 1
8 7 1 -- 2 3 -- 1 0 -- 2 4 1
2 4 1 -- 3 0 -- 1 0 -- 8 5 1
8 5 1 -- 2 0 -- 1 0 -- 2 3 1
2 3 1 -- 2 3 -- 1 0 -- 5 0 1
5 0 1 -- 2 0 -- 0 0 -- 8 1 1
episode: 72/10000, score: 6
[0.073584]
0.4657534246575342
*******************************
0 3 8 -- 2 0 -- 1 0 -- 3 4 8
3 4 8 -- 2 0 -- 1 0 -- 6 5 8
6 5 8 -- 2 0 -- 1 0 -- 0 3 8
0 3 8 -- 2 0 -- 1 0 -- 3 4 8
3 4 8 -- 2 0 -- 1 0 -- 6 5 8
6 5 8 -- 2 0 -- 1 0 -- 0 3 8
0 3 8 -- 2 0 -- 1 0 -- 3 4 8
3 4 8 -- 2 0 -- 1 0 -- 6 5 8
6 5 8 -- 2 0 -- 1 0 -- 0 3 8
0 3 8 -- 2 0 -- 1 0 -- 3 4 8
0.4594594594594595
*******************************
1 0 7 -- 2 0 -- 1 0 -- 4 1 7
4 1 7 -- 2 0 -- 0 1 -- 7 2 7
episode: 74/1

8 4 1 -- 2 3 -- 0 0 -- 2 1 1
episode: 105/10000, score: 9
[0.12605962]
0.42452830188679247
*******************************
3 5 0 -- 1 0 -- 1 0 -- 5 3 0
5 3 0 -- 0 0 -- 1 0 -- 3 4 0
3 4 0 -- 1 0 -- 1 0 -- 5 5 0
5 5 0 -- 0 0 -- 1 0 -- 3 3 0
3 3 0 -- 0 0 -- 1 0 -- 4 4 0
4 4 0 -- 2 0 -- 1 0 -- 7 5 0
7 5 0 -- 0 0 -- 1 0 -- 8 3 0
8 3 0 -- 2 0 -- 1 0 -- 2 4 0
2 4 0 -- 0 0 -- 0 0 -- 0 5 0
episode: 106/10000, score: 8
[0.1273183]
0.4205607476635514
*******************************
8 2 1 -- 2 0 -- 1 0 -- 2 0 1
2 0 1 -- 0 0 -- 0 1 -- 0 1 1
episode: 107/10000, score: 1
[0.12802164]
0.42592592592592593
*******************************
1 2 8 -- 2 0 -- 1 0 -- 4 0 8
4 0 8 -- 3 0 -- 1 0 -- 1 1 8
1 1 8 -- 2 0 -- 1 0 -- 4 2 8
4 2 8 -- 2 0 -- 1 0 -- 7 0 8
7 0 8 -- 0 3 -- 0 0 -- 8 6 8
episode: 108/10000, score: 4
[0.12941584]
0.42201834862385323
*******************************
1 3 4 -- 2 0 -- 0 0 -- 4 4 4
episode: 109/10000, score: 0
[0.12917548]
0.41818181818181815
*******************************
3 8 2 -- 2

4 1 7 -- 0 0 -- 1 0 -- 5 2 7
5 2 7 -- 0 0 -- 1 0 -- 3 0 7
3 0 7 -- 0 2 -- 1 0 -- 4 3 7
4 3 7 -- 2 0 -- 0 1 -- 7 4 7
episode: 145/10000, score: 8
[0.16532868]
0.4315068493150685
*******************************
1 7 8 -- 0 0 -- 0 0 -- 2 8 8
episode: 146/10000, score: 0
[0.16569707]
0.42857142857142855
*******************************
0 1 7 -- 0 0 -- 1 0 -- 1 2 7
1 2 7 -- 0 0 -- 1 0 -- 2 0 7
2 0 7 -- 0 0 -- 1 0 -- 0 1 7
0 1 7 -- 0 2 -- 1 0 -- 1 4 7
1 4 7 -- 0 0 -- 1 0 -- 2 5 7
2 5 7 -- 0 0 -- 1 0 -- 0 3 7
0 3 7 -- 0 0 -- 1 0 -- 1 4 7
1 4 7 -- 0 0 -- 1 0 -- 2 5 7
2 5 7 -- 0 3 -- 1 0 -- 0 2 7
0 2 7 -- 0 0 -- 1 0 -- 1 0 7
0.42567567567567566
*******************************
3 4 6 -- 0 0 -- 1 0 -- 4 5 6
4 5 6 -- 2 0 -- 1 0 -- 7 3 6
7 3 6 -- 0 0 -- 1 0 -- 8 4 6
8 4 6 -- 2 0 -- 1 0 -- 2 5 6
2 5 6 -- 0 0 -- 1 0 -- 0 3 6
0 3 6 -- 0 0 -- 1 0 -- 1 4 6
1 4 6 -- 0 0 -- 1 0 -- 2 5 6
2 5 6 -- 0 0 -- 1 0 -- 0 3 6
0 3 6 -- 0 0 -- 1 0 -- 1 4 6
1 4 6 -- 0 0 -- 1 0 -- 2 5 6
0.4228187919463087
*****************

...

3 8 6 -- 2 3 -- 0 0 -- 6 5 6
episode: 9736/10000, score: 2
[1.6400115]
0.2686659135257266
*******************************
2 3 6 -- 0 1 -- 1 0 -- 0 5 6
0 5 6 -- 2 1 -- 1 0 -- 3 4 6
3 4 6 -- 2 0 -- 0 0 -- 6 5 6
episode: 9737/10000, score: 2
[1.6394441]
0.26863832409118915
*******************************
7 8 6 -- 1 0 -- 0 0 -- 6 6 6
episode: 9738/10000, score: 0
[1.639445]
0.26861074032241505
*******************************
7 1 8 -- 0 0 -- 0 0 -- 8 2 8
episode: 9739/10000, score: 0
[1.6392616]
0.26858316221765915
*******************************
2 4 8 -- 0 0 -- 1 0 -- 0 5 8
0 5 8 -- 0 2 -- 0 0 -- 1 8 8
episode: 9740/10000, score: 1
[1.6388409]
0.2685555897751771
*******************************
3 7 6 -- 2 0 -- 0 0 -- 6 8 6
episode: 9741/10000, score: 0
[1.6386254]
0.2685280229932252
*******************************
8 3 0 -- 2 2 -- 1 0 -- 2 6 0
2 6 0 -- 0 1 -- 0 0 -- 0 8 0
episode: 9742/10000, score: 1
[1.6382388]
0.26850046187006055
*******************************
0 4 6 -- 2 0 -- 1 0 -- 3 5 

0 8 7 -- 0 1 -- 0 0 -- 1 7 7
episode: 9794/10000, score: 1
[1.6255721]
0.26778968861664115
*******************************
7 3 4 -- 2 1 -- 1 0 -- 1 5 4
1 5 4 -- 2 2 -- 0 0 -- 4 8 4
episode: 9795/10000, score: 1
[1.6251638]
0.26776235198040016
*******************************
8 0 1 -- 2 1 -- 1 0 -- 2 2 1
2 2 1 -- 0 3 -- 1 0 -- 0 8 1
0 8 1 -- 0 1 -- 0 0 -- 1 7 1
episode: 9796/10000, score: 2
[1.6245177]
0.2677350209247729
*******************************
6 3 4 -- 2 0 -- 0 0 -- 0 4 4
episode: 9797/10000, score: 0
[1.6243491]
0.2677076954480506
*******************************
2 0 6 -- 0 3 -- 0 0 -- 0 6 6
episode: 9798/10000, score: 0
[1.6241423]
0.26768037554852536
*******************************
6 7 1 -- 2 0 -- 1 0 -- 0 8 1
0 8 1 -- 0 1 -- 0 0 -- 1 7 1
episode: 9799/10000, score: 1
[1.6236559]
0.2676530612244898
*******************************
5 6 1 -- 0 3 -- 1 0 -- 3 3 1
3 3 1 -- 3 3 -- 1 0 -- 0 0 1
0 0 1 -- 0 3 -- 0 1 -- 1 6 1
episode: 9800/10000, score: 2
[1.625618]
0.26772778287929805
*

7 5 6 -- 1 1 -- 0 0 -- 6 4 6
episode: 9853/10000, score: 0
[1.6184542]
0.2670996549624518
*******************************
3 8 4 -- 0 2 -- 0 0 -- 4 2 4
episode: 9854/10000, score: 0
[1.6182988]
0.2670725520040588
*******************************
7 2 8 -- 0 0 -- 0 0 -- 8 0 8
episode: 9855/10000, score: 0
[1.6181226]
0.26704545454545453
*******************************
5 2 8 -- 0 3 -- 0 0 -- 3 8 8
episode: 9856/10000, score: 0
[1.6179632]
0.267018362584965
*******************************
4 2 3 -- 3 3 -- 1 0 -- 1 8 3
1 8 3 -- 1 3 -- 1 0 -- 0 5 3
0 5 3 -- 0 0 -- 0 0 -- 1 3 3
episode: 9857/10000, score: 2
[1.6174666]
0.266991276120917
*******************************
8 4 7 -- 0 0 -- 1 0 -- 6 5 7
6 5 7 -- 0 0 -- 0 0 -- 7 3 7
episode: 9858/10000, score: 1
[1.6171657]
0.2669641951516381
*******************************
2 4 8 -- 0 0 -- 1 0 -- 0 5 8
0 5 8 -- 0 2 -- 0 0 -- 1 8 8
episode: 9859/10000, score: 1
[1.6168365]
0.2669371196754564
*******************************
4 3 2 -- 1 2 -- 1 0 -- 3 6 2
3 

0 6 7 -- 0 0 -- 0 0 -- 1 7 7
episode: 9912/10000, score: 2
[1.6193386]
0.2667204680722284
*******************************
5 1 3 -- 0 3 -- 0 0 -- 3 7 3
episode: 9913/10000, score: 0
[1.6190894]
0.26669356465604194
*******************************
4 5 3 -- 3 0 -- 0 0 -- 1 3 3
episode: 9914/10000, score: 0
[1.6188766]
0.26666666666666666
*******************************
6 5 3 -- 2 0 -- 0 0 -- 0 3 3
episode: 9915/10000, score: 0
[1.6186727]
0.26663977410246065
*******************************
6 2 5 -- 2 3 -- 1 0 -- 0 8 5
0 8 5 -- 1 1 -- 1 0 -- 2 7 5
2 7 5 -- 2 0 -- 0 0 -- 5 8 5
episode: 9916/10000, score: 2
[1.6181145]
0.2666128869617828
*******************************
2 7 6 -- 0 0 -- 1 0 -- 0 8 6
0 8 6 -- 0 0 -- 0 0 -- 1 6 6
episode: 9917/10000, score: 1
[1.6177304]
0.26658600524299253
*******************************
1 5 4 -- 2 2 -- 0 0 -- 4 8 4
episode: 9918/10000, score: 0
[1.617546]
0.26655912894445005
*******************************
1 3 4 -- 2 0 -- 0 0 -- 4 4 4
episode: 9919/10000, score

4 0 2 -- 3 3 -- 1 0 -- 1 6 2
1 6 2 -- 0 0 -- 0 0 -- 2 7 2
episode: 9975/10000, score: 1
[1.6140862]
0.2660384923817161
*******************************
0 1 5 -- 0 3 -- 1 0 -- 1 7 5
1 7 5 -- 1 0 -- 1 0 -- 0 8 5
0 8 5 -- 0 3 -- 0 0 -- 1 5 5
episode: 9976/10000, score: 2
[1.6138035]
0.2660118272025659
*******************************
2 3 6 -- 0 2 -- 0 0 -- 0 6 6
episode: 9977/10000, score: 0
[1.6136509]
0.26598516736821004
*******************************
5 7 8 -- 2 0 -- 0 0 -- 8 8 8
episode: 9978/10000, score: 0
[1.6135261]
0.2659585128770418
*******************************
2 0 6 -- 0 3 -- 0 0 -- 0 6 6
episode: 9979/10000, score: 0
[1.6133378]
0.2659318637274549
*******************************
4 5 2 -- 1 2 -- 1 0 -- 3 8 2
3 8 2 -- 2 2 -- 0 0 -- 6 2 2
episode: 9980/10000, score: 1
[1.6130043]
0.2659052199178439
*******************************
8 1 3 -- 2 3 -- 1 0 -- 2 7 3
2 7 3 -- 0 0 -- 1 0 -- 0 8 3
0 8 3 -- 0 3 -- 1 0 -- 1 5 3
1 5 3 -- 1 0 -- 0 0 -- 0 3 3
episode: 9981/10000, score: 3
[1.61

In [ ]:
class A2CAgent:
    def __init__(self, state_size, action_size):
        # if you want to see Cartpole learning, then change to True
        self.render = False
        self.load_model = False
        # get size of state and action
        self.state_size = state_size
        self.action_size = action_size
        self.value_size = 1
        #self.action_table = [0,1,-4,4]
        self.lam = 0
        self.alpha = 0.2
        #self.target_state = 11

        # These are hyper parameters for the Policy Gradient
        self.discount_factor = 1
        self.actor_lr = 0.005
        self.critic_lr = 0.005

        # create model for policy network
        self.actor_red = self.build_actor()
        self.actor_blue = self.build_actor()
        self.critic = self.build_critic()
        self.critic2 = self.build_critic2()

        #if self.load_model:
            #self.actor.load_weights("./save_model/cartpole_actor.h5")
            #self.critic.load_weights("./save_model/cartpole_critic.h5")

    # approximate policy and value using Neural Network
    # actor: state is input and probability of each action is output of model
    def build_actor(self):
        actor = Sequential()
        actor.add(Dense(48, input_dim=self.state_size, activation='relu',
                        kernel_initializer='he_uniform'))
        #actor.add(Dense(16, activation='relu',
                        #kernel_initializer='he_uniform'))
        actor.add(Dense(self.action_size, activation='softmax',
                        kernel_initializer='he_uniform'))
        actor.summary()
        # See note regarding crossentropy in cartpole_reinforce.py
        actor.compile(loss='categorical_crossentropy',
                      optimizer=Adam(lr=self.actor_lr))
        return actor

    # critic: state is input and value of state is output of model
    def build_critic(self):
        critic = Sequential()
        critic.add(Dense(48, input_dim=self.state_size, activation='relu',
                         kernel_initializer='he_uniform'))
        #critic.add(Dense(16, activation='relu',
                         #kernel_initializer='he_uniform'))
        critic.add(Dense(self.value_size, activation='linear',
                         kernel_initializer='he_uniform'))
        #critic.summary()
        critic.compile(loss="mse", optimizer=Adam(lr=self.critic_lr))
        return critic
    
    def build_critic2(self):
        critic = Sequential()
        critic.add(Dense(48, input_dim=self.state_size, activation='relu',
                         kernel_initializer='he_uniform'))
        #critic.add(Dense(16, activation='relu',
                         #kernel_initializer='he_uniform'))
        critic.add(Dense(self.value_size, activation='linear',
                         kernel_initializer='he_uniform'))
        #critic.summary()
        critic.compile(loss="mse", optimizer=Adam(lr=self.critic_lr))
        return critic
        

    # using the output of policy network, pick action stochastically
    def get_action(self, state):
        policy1 = self.actor_red.predict(state, batch_size=1).flatten()
        policy2 = self.actor_blue.predict(state,batch_size=1).flatten()
        return np.random.choice(self.action_size, 1, p=policy1)[0],np.random.choice(self.action_size, 1, p=policy2)[0]
    
    def get_maxaction(self,state): #To be updated
        policy1 = self.actor_red.predict(state, batch_size=1).flatten()
        policy2 = self.actor_blue.predict(state,batch_size=1).flatten()
        return np.argmax(policy1),np.argmax(policy2)

    # update policy network every episode
    def train_model(self, state, action1,action2, reward1,reward2, next_state, done):
        target = np.zeros((1, self.value_size))
        advantages_red = np.zeros((1, self.action_size))
        advantages_blue = np.zeros((1, self.action_size))
        target2 = np.zeros((1, self.value_size))     

        value = self.critic.predict(state)[0]
        value2 = self.critic2.predict(state)[0]
        next_value = self.critic.predict(next_state)[0]
        next_value2 = self.critic2.predict(next_state)[0]
        
        reward1 = reward1 + self.lam*(reward2 - self.alpha)

        if done:
            advantages_red[0][action1] = -1*(reward1 - value)
            advantages_blue[0][action2] = -1*(reward1 - value)
            target[0][0] = reward1
            target2[0][0] = reward2
        else:
            advantages_red[0][action1] = -1*(reward1 + self.discount_factor * (next_value) - value)
            advantages_blue[0][action2] = -1*(reward1 + self.discount_factor * (next_value) - value)
            target[0][0] = reward1 + self.discount_factor * next_value
            target2[0][0] = reward2 + self.discount_factor * next_value2
#             if np.where(next_state == 1.0)[1][0] == self.target_state:
#                 target2[0][0] = 0 + self.discount_factor * next_value2
#             else:
#                 target2[0][0] = np.where(next_state == 1.0)[1][0]/10 + self.discount_factor * next_value2            
            

        self.actor_red.fit(state, advantages_red, epochs=1, verbose=0)
        self.actor_blue.fit(state, advantages_blue, epochs=1, verbose=0)
        self.critic.fit(state, target, epochs=1, verbose=0)
        self.critic2.fit(state,target2,epochs = 1, verbose = 0)
        
        #initial_state = np.zeros((1,state_size))
        #initial_state[0][0] = 1
        self.lam = min(max(0,(self.lam + 0.001*(self.critic2.predict(state)[0] - self.alpha))), 10)
        #print(self.lam)
        

if __name__ == "__main__":
    # In case of CartPole-v1, maximum length of episode is 500
    #env = gym.make('CartPole-v1')
    # get size of state and action from environment
    state_size = 4*3*3 #env.observation_space.shape[0]
    action_size = 4 #env.action_space.n
    EPISODES = 20000
    
    np.random.seed(123)
    random.seed(456)

    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)


    tf.set_random_seed(789)

    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)
    
    

    # make A2C agent
    agent = A2CAgent(state_size, action_size)
    #agent2 = A2CAgent(state_size,action_size)
    game = CoinGameVec()
    
    scores, episodes = [], []

    for i in range(1):
        done = False
        score = 0
        #state = env.reset()
        #state = np.reshape(state, [1, state_size])
        #rand_state = random.randint(0,15)
        #state = np.zeros((1,state_size))
        #state[0][rand_state] = 1
        #state = np.reshape(state, [1, state_size])
        
        average_penalty = 0
        total_penalty = 0
        store_penalty = np.zeros((1,EPISODES))
        
        average_lag = 0
        total_lag = 0
        store_lag = np.zeros((1,EPISODES))

        
        for e in range(EPISODES):
        #state = env.reset()
            state = game.reset()
            state = np.reshape(state, [1, state_size])
            for time in range(10):
            # env.render()
                action1,action2  = agent.get_action(state)
                
                next_state, reward, done = game.step(action1,action2)
                
            #reward = reward if not done else -10
                next_state = np.reshape(next_state, [1, state_size])
                agent.train_model(state, action1, action2,reward[0][0],reward[1][0],next_state, done)
                #agent2.train_model(state, action2, reward[0][0],reward[1][0],next_state, done)
                
                
                print(np.where(state == 1.0)[1][0],np.where(state == 1.0)[1][1] - 9,int(np.where(state == 1.0)[1][2] % 9),"--",action1, action2,"--",reward[0][0],reward[1][0],"--",np.where(next_state == 1.0)[1][0],np.where(next_state == 1.0)[1][1] - 9,np.where(next_state == 1.0)[1][2] % 9)
                
                #print(agent1.lam,agent2.lam)
                state = next_state
                
                average_penalty = average_penalty + reward[1][0]
                average_lag = average_lag + reward[0][0]
                
                if done:
                #print(state)
                    print("episode: {}/{}, score: {}"
                      .format(e, EPISODES, time))
                    print(agent.lam)
                    #print(agent2.lam)
                    break
                #if len(agent.memory) > batch_size:
                    #agent.replay(batch_size)
            total_penalty = total_penalty + average_penalty        
            store_penalty[0][e] = total_penalty/(e+1) 
            print(store_penalty[0][e])
                
            total_lag = total_lag + average_lag
            store_lag[0][e] = total_lag/(e+1)
                
                #if np.mean(store_penalty[0][e-100:e]) < agent.alpha+0.1:
                        #sys.exit()
                #print(store_penalty[0][e])
            average_penalty = 0
            average_lag = 0
        
            print('*******************************')

In [ ]:
state = game.reset()
state = np.reshape(state, [1, state_size])
print(state)
policy1 = agent.actor_red.predict(state, batch_size=1).flatten()
policy2 = agent.actor_blue.predict(state,batch_size=1).flatten()
print(agent.critic2.predict(state)[0])
#print(policy1,policy2)


In [9]:
np.random.seed(99)
random.seed(999)
prng.seed(999)

count = 0
total = 10000
for i in range(total):
    state = game.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(10):
                # env.render()
        action1,action2 = agent.get_maxaction(state)
        #action2 = agent2.get_action(state)

        next_state, reward, done = game.step(action1,action2)

                #reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])  
        #print(np.where(state == 1.0)[1][0],np.where(state == 1.0)[1][1] - 9,np.where(state == 1.0)[1][2],"--",action1, action2,"--",reward[0][0],reward[1][0],"--",np.where(next_state == 1.0)[1][0],np.where(next_state == 1.0)[1][1] - 9,np.where(next_state == 1.0)[1][2])

                    #print(agent1.lam,agent2.lam)
        state = next_state
        #print(agent.critic2.predict(state)[0])
        if done:
                    #print(state)
            #print("episode: {}/{}, score: {}"
                    #.format(e, EPISODES, time))
            #print(agent.lam)
            #print(agent2.lam)
            if reward[1][0] == 1:
                count = count+1
            
            break
                    #if len(agent.memory) > batch_size:
                        #agent.replay(batch_size)

                   #print('*******************************')
print(count/total)

0.1388


In [12]:
np.savetxt('CGCentralized_penalty',store_penalty)
np.savetxt('CGCentralized_Lag',store_lag)

In [10]:
print(store_lag)

[[0.         1.5        1.66666667 ... 1.26165233 1.26162616 1.2617    ]]
